In [3]:
from skimage import io
import numpy as np
import cv2
from skimage.util import img_as_bool, img_as_float, pad
from skimage.color import gray2rgb, rgb2gray
from skimage.morphology import square, closing, disk, erosion, dilation, diamond
from skimage.draw import line
from skimage.transform import rotate
from skimage.measure import regionprops
import itertools
import math
import uuid

%matplotlib inline

def remove_not_center_lines(img):
    img = closing(img, square(3))
    cy = int((img.shape[0]-1)/2)
    cx = int((img.shape[1]-1)/2)
    img[cy-1:cy+1,cx-1:cx+1] = 255
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity = 8)
    img[labels!=labels[cy,cx]] = 0
    return img

def leave_one_edge_point(line):
    m = line.max()
    a_m = np.argmax(line)
    if line[0] == m and line[len(line) -1] == m:
        line[1:len(line) -1] = 0
        return
    if line[a_m] == m:
        line[:]=0
        line[a_m]=m
  
def get_edge_points(img):
    idx = np.where(img > 0)
    a,b = idx
    idx = np.array(list(zip(a,b)))
    return idx

def find_edge_points(img):
    img = remove_not_center_lines(img)
    
    h,w = img.shape
    img[1:h-1,1:w-1] = 0
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity = 8)
    for i in range(1, labels.max()+1):
        idx = np.where(labels == i)
        y, x = idx[0][0],idx[1][0]
        img[labels == i] = 0
        img[y,x] = 255
    return get_edge_points(img)


def calc_angle_4(x1,y1, x2,y2):
    angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
    return angle

def calc_angle(p1, p2):
    x1,y1 = p1
    x2,y2 = p2
    angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
    #angle = 360+angle if angle < 0 else angle
    return angle
def findAngle(p1,c,p2):
    x1,y1 = p1
    x2,y2 = p2  
    xc,yc = c
    a = math.pow(xc-x1,2) + math.pow(yc-y1,2)
    b = math.pow(xc-x2,2) + math.pow(yc-y2,2)
    c = math.pow(x2-x1,2) + math.pow(y2-y1,2)
    return math.degrees(math.acos( (a+b-c) / math.sqrt(4*a*b) ))
    
def filter_by_filling(points, img):
    result = []
    for x,y in points:
        w = 30
        img_w = img[y-w:y+w+1, x-w:x+w+1]
        non_z = np.count_nonzero(img_w)
        if non_z/img_w.size < 0.40 and non_z/img_w.size > 0.15:
            result.append((x,y))
    return result


def section_len(x1,y1,x2,y2):
    return math.sqrt((x2-x1)**2+(y2-y1)**2)

def get_longest_line(lines):
    max_len = 0
    longest_line = None
    if len(lines) == 1:
        return np.squeeze(lines)
    for x1,y1,x2,y2 in lines:
        s_len = section_len(x1,y1,x2,y2)
       #print(x1,y1,x2,y2, "len:", s_len)
        if(s_len > max_len):
            longest_line = (x1,y1,x2,y2)
            max_len = s_len
    return longest_line
    


def filter_non_90(points,img):
    tmp = []
    w_s = 25
    result = []

    for x,y in points:
        img_w = img[y-w_s:y+w_s+1, x-w_s:x+w_s+1]
        edges = cv2.Canny(img_w,0,255)  
        if edges is None:
            print("warn no edges")
            continue
        #edges = closing(edges, square(3))
        #io.imsave(str(x) + str(y) + ".png", edges)
        h,w = edges.shape
        img_w_copy = np.copy(edges)
        idx = find_edge_points(img_w_copy)
        cx = int((h-1)/2)
        cy = int((w-1)/2)
        if len(idx) != 2:
            #io.imsave(str(x) + str(y) + ".png", edges)
            continue
        
        cx = int((h-1)/2)
        cy = int((w-1)/2)
        a0 = calc_angle(idx[0],(cx,cy))
        a1 = calc_angle(idx[1],(cx,cy))
        angle = findAngle(idx[0], (cx,cy), idx[1])
        #print("x,y idx[0], (cx,cy), idx[1]",(x,y), idx[0], (cx,cy), idx[1])
        #print(angle, a1,a0) 
        if angle < 110 and angle > 70:
            result.append((x,y))
        #tmp.append({"points":(x,y), "angle":angle})
            
    #result = []
    #for i in tmp:
    #    for j in tmp:
    #        if(i==j):
    #            continue
    #        if(i["angle"] + j["angle"] < 185 and i["angle"] + j["angle"] > 175):
    #            result.append(i["points"])
    #            result.append(j["points"])
    return result


def process(img):
    img = img.astype("uint8")
    edges = cv2.Canny(img,0,255)
    edges = dilation(edges, square(3))
    #edges = edges > 0
    return edges
    
    img = pad(img,(30,30),'constant', constant_values=(0, 0))
    img_d = np.copy(img)    
    img_d = gray2rgb(img_d)

    dst = cv2.cornerHarris(img,2,3,0.04)
    ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
    dst = np.uint8(dst)
    # Threshold for an optimal value, it may vary depending on the image.
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    centroids = np.int0(centroids)
    a = centroids[:,0]
    b = centroids[:,1]
    points = list(zip(a,b))

    
    for x,y in points:
        img_d[y-12:y+12,x-12:x+12] = (155,255,155)
    #print("before candidates : {}".format(len(points)))
    
    points = filter_by_filling(points,img)
    for x,y in points:
        img_d[y-10:y+10,x-10:x+10] = (0,255,0)
    #print("after filling candidates : {}".format(len(points)))
        
    points = filter_non_90(points,img)
    #print("after non 90 candidates : {}".format(len(points)))
    for x,y in points:
        img_d[y-7:y+7,x-7:x+7] = (0,0,255)
      
    edges = cv2.Canny(img,0,255)
    edges = dilation(edges, square(5))
    edges = edges > 0
    lines = []
    for x1,y1 in points:
        for x2,y2 in points:
            if x1 == x2 and y1 == y2:
                continue
            p1_p2_line = line(y1,x1, y2,x2)
            is_ok = np.all(edges[p1_p2_line])
            if is_ok:
                    lines.append((x1,y1,x2,y2))
    if len(lines) > 0:
        x1,y1, x2,y2 = get_longest_line(lines)
        img_d[y1-5:y1+5,x1-5:x1+5] = (255,0,0)
        img_d[y2-5:y2+5,x2-5:x2+5] = (255,0,0)
        angle = calc_angle_4(x1,y1,x2,y2)
        img_d = rotate(img_d,angle, resize = True)
        img = rotate(img, angle, resize = True)

        h,w = img.shape
        upper_half = img[0:int(h/2),:]
        bottom_half = img[int(h/2):h,:]
        if np.count_nonzero(upper_half) > np.count_nonzero(bottom_half):
            img = rotate(img,180, resize = True)
            img_d = rotate(img_d,180, resize = True)
        
        img_mask = np.copy(img)    
        img_mask[img_mask>0] = 1
        img_mask = img_mask.astype("uint8")
        
        edges = cv2.Canny(img_mask,0,1)
        edges = dilation(edges)
        
        
        region = regionprops(edges)[0]
        minr, minc, maxr, maxc = region.bbox
        img = edges[minr:maxr-5, minc+10:maxc-10]
        
        region = regionprops(img)[0]
        minr, minc, maxr, maxc = region.bbox
        img = img[minr:maxr, minc:maxc]

        #edges = edges > 0 
        #for i, value in enumerate(img[::-1]):
            #if np.any(value == 0):
                #img = img[0:i+30]
    return img

img = io.imread("data/set9/0.png")
img = process(img)
io.imshow(img)



ImportError: No module named 'cv2'

In [18]:
import glob
for filename in glob.iglob('data/set9/*[!pa].png', recursive=True):
    print(filename)
    img = io.imread(filename)
    img = process(img)
    print(filename)
    data_dir, set_dir, f = filename.split('/')
    f, e = f.split('.')
    io.imsave(data_dir + "/" + set_dir + "/" + f + "p.png", img)


data/set9/295.png
data/set9/295.png
data/set9/4944.png
data/set9/4944.png
data/set9/8408.png
data/set9/8408.png
data/set9/4152.png
data/set9/4152.png
data/set9/111.png
data/set9/111.png
data/set9/7571.png
data/set9/7571.png
data/set9/1652.png
data/set9/1652.png
data/set9/6307.png
data/set9/6307.png
data/set9/6664.png
data/set9/6664.png
data/set9/8249.png
data/set9/8249.png
data/set9/9973.png
data/set9/9973.png
data/set9/118.png
data/set9/118.png
data/set9/7252.png
data/set9/7252.png
data/set9/8387.png
data/set9/8387.png
data/set9/9236.png
data/set9/9236.png
data/set9/4365.png
data/set9/4365.png
data/set9/4208.png
data/set9/4208.png
data/set9/7373.png
data/set9/7373.png
data/set9/1984.png
data/set9/1984.png
data/set9/7070.png
data/set9/7070.png
data/set9/8532.png
data/set9/8532.png
data/set9/6823.png
data/set9/6823.png
data/set9/5234.png
data/set9/5234.png
data/set9/3513.png
data/set9/3513.png
data/set9/5364.png
data/set9/5364.png
data/set9/4247.png
data/set9/4247.png
data/set9/2011.png